# Final Project: Phase 2: SQL Cleanup

####Import key libraries and data file with information to connect to MySQL

In [1]:
import numpy as np
import pandas as pd
import MySQLdb as mdb
import MySQL_data_file as MySQL_data

####Create connection object "con" for accessing databases:

In [2]:
con = mdb.connect(MySQL_data.my_sql_host, MySQL_data.my_sql_user,\
                    MySQL_data.my_sql_passwd,\
                    MySQL_data.my_sql_database)
#con.close

####Define key functions for querying information from MySQL databases:

In [3]:
def table_information(table_name, columns_only=False):
    """
    Function returns table information. Will return list of columns if columns_only is set
    """
    cur = con.cursor()
    cur.execute("DESCRIBE {}".format(table_name))
    if columns_only == False:
        return [x for x in cur.fetchall()]
    else:
        #column_list = []
        return [row[0:1][0] for row in cur.fetchall()]      
#table_information('mmaps_data', True)

In [4]:
def run_sql(query):
    cur = con.cursor()
    cur.execute(query)
    con.commit()
    #con.close()
    return cur.fetchall()

####Initial Test Queries

In [29]:
#run_sql("""SHOW TABLES""")
#run_sql("""SELECT datetime,origins,destinations,duration FROM gmaps_data""")
#table_count = run_sql("""SELECT COUNT(*) FROM mmaps_data""")
#table_count

((19080L,),)

####Adjusting Date and Time Formats in Python

In [321]:
#Converting datetime to ISO format in Python
sample_dt_object = run_sql("""SELECT datetime FROM gmaps_data limit 1""")
sample_dt_object_iso = sample_dt_object[0][0].isoformat(" ")
#sample_dt_object_iso

####Adjusting Date and Time Formats in SQL

In [323]:
#Working with timestamps and time functions in SQL
#run_sql("""SELECT CURRENT_TIMESTAMP""")#or NOW() or CURRENT_TIMESTAMP()
#run_sql("""SELECT HOUR(datetime) FROM gmaps_data limit 10""")
#run_sql("""SELECT UNIX_TIMESTAMP(datetime) FROM gmaps_data limit 10""")
#run_sql("""SELECT GET_FORMAT(datetime, 'ISO') FROM gmaps_data limit 1""")

In [325]:
#retrieving time data in ISO format (through SQL)
#run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s') FROM gmaps_data limit 3""")
#converting to PST time zone
#run_sql("""SELECT CONVERT_TZ(DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),'+00:00','-07:00') FROM gmaps_data limit 1""")

In [10]:
#NOTE: this is an effective way to convert to ISO format
run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE datetime BETWEEN '2015-05-02 11:00:00' AND '2015-05-02 11:14:00'
        AND origins LIKE 'Pac%'
        LIMIT 1
        """)

(('2015-05-02 11:00:02',
  'Pacific Heights, San Francisco, CA',
  'Oakland, CA',
  2005L),)

####Two different ways to look at the same route between two locations (morning commute and afternoon return commutes

<p>The code below shows Google Maps transit data between 4am and 11am on Saturday May 2, traveling from Pac Heights to Financial District</p>

In [129]:
#The code below shows transit data between 4am and 11am on Saturday May 2 
#traveling from Pac Heights to Financial District
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
            ((datetime BETWEEN '2015-05-01 11:00:00' AND '2015-05-01 18:01:00')
            OR 
            (datetime BETWEEN '2015-05-01 21:00:00' AND '2015-05-02 04:01:00'))
        AND 
            ((origins LIKE 'Pac%' AND destinations LIKE 'Fin%')
            OR 
            (origins LIKE 'Fin%'and destinations LIKE 'Pac%'))
        ORDER BY datetime
        """))

58

<p>The code below is a different style but produces the same result as the SQL statement above.<br />
It shows transit data between 4am and 11am on Saturday May 2, traveling from Pac Heights to Financial District</p>

In [32]:
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
        ((datetime BETWEEN '2015-05-01 11:00:00' AND '2015-05-01 18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%')) 
        OR
        ((datetime BETWEEN '2015-05-01 21:00:00' AND '2015-05-02 04:01:00')
        AND (origins LIKE 'Fin%') AND (destinations LIKE 'Pac%')) 
        ORDER BY datetime
        """))

58

####Ordering Results by Hours/Minutes/Seconds in order to view travel duration values from multiple days in consecutive rows:

<p>The following SQL command examines a specific route (and its return trip), ordering the results by hours/minutes/seconds, effectively mapping multiple days worth of data onto a single 24 hour scale. At certain intervals in this 24 hour period (e.g. every 15 minutes from 4AM to 11AM and 2AM to 9PM), we have multiple datapoints (travel duration values)--each representing a specific day in which data was collected.<br />
<p>NOTE: Due to UTC formatting, the code below displays a route (and its return trip) out of order. In order to set everything chronologically, one would need to locally adjust the time zone.</p>

In [7]:
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%')) 
        OR
        ((TIME(datetime) BETWEEN '21:00:00' AND '24:00:00' OR
        (TIME(datetime) BETWEEN '00:00:00' AND '04:01:00'))
        AND (origins LIKE 'Fin%') AND (destinations LIKE 'Pac%'))
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        """))

704

Given the UTC time zone issue above, I've decided to set the following code to display morning driving times only (not the return trip) for the commute from pac heights to the financial district in SF.

In [35]:
#Took the code from above and specified the query such that we are looking
#at the morning driving times for the commute from pac heights to financial district
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        """))

377

####Normalizing the strings used to represent origins/destinations for each API (Google and Bing)

<p>The following checks to see how the origins and destinations are represented for the two APIs</p>

In [45]:
#run_sql("""select DISTINCT origins from gmaps_data""")
#run_sql("""select DISTINCT destinations from gmaps_data""")
#run_sql("""select DISTINCT origins from mmaps_data""")
#run_sql("""select DISTINCT destinations from mmaps_data""")

<p>The following updates the MySQL tables, effectively normalizing origin and destination string values</p>

In [53]:
'''
run_sql("""UPDATE gmaps_data
        SET destinations=(SELECT DISTINCT destinations FROM mmaps_data WHERE destinations LIKE 'North%')
        WHERE destinations LIKE 'North%'
        """)
'''
print "NOTE: Intend to normalize location string values using pandas (although it is possible with SQL)"

NOTE: Intend to normalize location string values using pandas (although it is possible with SQL)


####Joining tables with SQL (union including all values from both tables)

<P>NOTE: Using an SQL Union is not preferrable because you cannot ORDER BY datetime without error</P>

In [270]:
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic,travel_mode
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        UNION ALL
        SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration,travel_mode
        FROM gmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        """))

464

####Using AVG() AND GROUP BY  to see average travel times at each interval

In [55]:
run_sql("""SELECT DATE_FORMAT(datetime,'%H:%i') as datetime, origins, destinations, AVG(duration_traffic) / 60 as MINUTES
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Berk%') AND (destinations LIKE 'Moun%'))
        GROUP BY HOUR(datetime), MINUTE(datetime)
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        LIMIT 1
        """)

(('11:00', 'Berkeley, CA', 'Mountain View, CA', Decimal('58.75128205')),)

####Reading SQL into pandas DataFrame (example)

In [57]:
test_df = pd.read_sql("""SELECT DATE_FORMAT(datetime,'%H:%i') as datetime, origins, destinations, AVG(duration_traffic) / 60 as MINUTES
        FROM mmaps_data
        WHERE 
        (TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        GROUP BY origins,destinations,HOUR(datetime), MINUTE(datetime)
        ORDER BY HOUR(datetime), MINUTE(datetime),origins,destinations
        """, con)
test_df.head(5)

,datetime,origins,destinations,MINUTES
0,11:00,"Berkeley, CA","Financial District, San Francisco, CA",26.282051
1,11:00,"Berkeley, CA","Mountain View, CA",58.751282
2,11:00,"Berkeley, CA","Oakland, CA",16.617949
3,11:00,"Mission District, San Francisco, CA","Financial District, San Francisco, CA",22.721795
4,11:00,"Mission District, San Francisco, CA","Mountain View, CA",42.869231


####Reading and Combining Two SQL Tables Into pandas DataFrame

In [98]:
gmaps_df = pd.read_sql("""SELECT entry_id,timestamp,datetime,origins,destinations,travel_mode,duration,distance,fare
                        FROM gmaps_data""",con)
gmaps_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29628 entries, 0 to 29627
Data columns (total 9 columns):
entry_id        29628 non-null int64
timestamp       29628 non-null datetime64[ns]
datetime        29628 non-null datetime64[ns]
origins         29628 non-null object
destinations    29628 non-null object
travel_mode     29628 non-null object
duration        29454 non-null float64
distance        29454 non-null float64
fare            17128 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(3)
memory usage: 2.3+ MB


In [89]:
bing_df = pd.read_sql("""SELECT entry_id,timestamp,datetime,origins,destinations,travel_mode,duration_traffic,distance,congestion
                        FROM mmaps_data""",con)
bing_df.info()   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19080 entries, 0 to 19079
Data columns (total 9 columns):
entry_id            19080 non-null int64
timestamp           19080 non-null datetime64[ns]
datetime            19080 non-null datetime64[ns]
origins             19080 non-null object
destinations        19080 non-null object
travel_mode         19080 non-null object
duration_traffic    19080 non-null int64
distance            19080 non-null int64
congestion          19080 non-null object
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 1.5+ MB


In [90]:
#changing the bing maps "duration_traffic" column to "duration" in order to match up the column with the gmaps_df
bing_df.columns = [u'entry_id', u'timestamp', u'datetime', u'origins', u'destinations', u'travel_mode', u'duration', u'distance', u'congestion']

In [91]:
combined_df = gmaps_df.merge(bing_df, how='outer')

In [92]:
combined_df = combined_df.copy()

In [93]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48708 entries, 0 to 48707
Data columns (total 10 columns):
entry_id        48708 non-null float64
timestamp       48708 non-null datetime64[ns]
datetime        48708 non-null datetime64[ns]
origins         48708 non-null object
destinations    48708 non-null object
travel_mode     48708 non-null object
duration        48534 non-null float64
distance        48534 non-null float64
fare            17128 non-null float64
congestion      19080 non-null object
dtypes: datetime64[ns](2), float64(4), object(4)
memory usage: 4.1+ MB
